# <center>HW 3: Scrape Products </center>

- Q1. Write a function `getProducts(page_url, domain_name, limit = 100)` to scrape all **product inforamtion**, including **product description** (see (1) in Figure) and **price** (see (2) in Figure).
    * Inputs:
        - `page_url`: the starting URL, i.e. 'http://www.pricegrabber.com/computer-monitors/browse/'
        - `domain_name`: website domain name, i.e. 'http://www.pricegrabber.com'
        - `limit`: the minimum number of products to be scraped
    * `Output`: 
        - a DataFrame saved all scraped products with columns [`name`, `price`]
    * Note:
        * If a field, e.g. price, is missing, use `None` to indicate it.
        * When you need to scrape the next page, your program should scrape the target of the `next` link (see (3) in Figure) for next page, and then send a request to this target to keep scraping. 
        * Stop scrapping if you have scraped the number of pages as specified by `limit`. For example, if `limit=3` and each page has 20 products, you'll scrape 60 products. If the total number of pages is less than `limit`, then you scrape all pages.
    
    <img src='Q1.png' width='60%'>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# add your import statements

In [2]:
def getProducts(page_url, domain_name, limit=3):    
    page_number=0
    name=None
    price=None
    data_name=[]
    data_price=[]
    result=None
    while page_number<limit:
        page = requests.get(page_url)
        soup = BeautifulSoup(page.content, 'html.parser')
        prices=soup.select("div.ctaPriceContainer")
        names=soup.select("div.details.clearfix")
        if page.status_code==200:
            for price in prices:
                price1=price.select("a.productPrice")
                if price1!=[]:
                    price=price1[0].get_text().strip()
                data_price.append(price)
            for name in names:
                name1=name.select("a.resultsListTitle")
                if name1!=[]:
                    name=name1[0].get_text().strip()
                data_name.append(name)
            next_page = soup.select("a.next")                      
            if next_page!=[]:
                page_url = domain_name + next_page[0]["href"]
                
            page_number +=1
    data_price=pd.DataFrame(data_price,columns=['price'])
    data_name=pd.DataFrame(data_name,columns=['name'])
    result=pd.concat([data_name,data_price],axis=1)


    return result
    

                


In [3]:
domain_name = 'http://www.pricegrabber.com'
page_url = 'http://www.pricegrabber.com/computer-monitors/browse/'

products = getProducts(page_url, domain_name, limit=2)
print(len(products))
print(products.head(5))

40
                                               name      price
0  Duracell 76355 - DURACELL HIGH POWER INVERTER...     $49.99
1  TVLogic LVM-170A 17" Full HD SD/HD/3G-SDI Mul...  $1,975.00
2  Planar Systems Universal Height Adjust Stand ...     $69.00
3  ikan VXF2-V2 4K HDMI/3G-SDI On-Camera Monitor...    $699.99
4  Lenovo ThinkVision Creator Extreme 27" UHD Mi...  $3,534.00


- Q2 (Bonus). Modify the function you defined in Q1 to scrape the products that are <font color=red>only in specific product line</font>. Name this new function as `getFilteredProducts(driver, page_url, domain_name, product_line, limit = 2)`. 
- Inputs:
    - `drive`: a initialized Selenium Drive instance
    - `page_url`: the starting URL, i.e. http://www.pricegrabber.com/computer-monitors/browse/
    - `domain_name`: website domain name, i.e. http://www.pricegrabber.com
    - `product_line`: a product line specified by user, e.g. 'ThinkVision'
    - `limit`: maximum number of pages to be scraped 
- Outputs: 
    - the scraped products as in Q1.
- Note:
    - You have to use your program to choose the specified product line from the left panel (see (4) in the figure), and then scrape products as you did in Q1. 

<img src='Q2.png' width='60%'>

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions 
def getFilteredProducts(driver, page_url, domain_name, product_line='ThinkVision', limit=100):
    products=None
    executable_path = 'driver/chromedriver'
    driver = webdriver.Chrome(executable_path=executable_path)
    driver.get('http://www.pricegrabber.com/computer-monitors/browse/')
    driver.find_element_by_css_selector('a[href*="flexscan"]').click()
    u=driver.current_url
    products= getProducts(u, domain_name, limit)

    return products

In [7]:
# change driver by web browser
#Options = webdriver.chrome.options.Options()
#Options.headless = True
#driver = webdriver.Chrome()

driver = webdriver.Chrome()

driver.implicitly_wait(3)  # set implict wait

domain_name = 'http://www.pricegrabber.com'
page_url = 'http://www.pricegrabber.com/computer-monitors/browse/'


products = getFilteredProducts(driver, page_url, domain_name, 
                               product_line='flexscan', limit=3)

driver.quit()
products

,name,price
0,"EIZO FlexScan EV3285 31.5"" 16:9 IPS Monitor (...","$1,839.00"
1,"EIZO FlexScan EV3895FX-BK 37.5"" 24:10 Ultrawi...","$1,889.00"
2,"EIZO FlexScan S1934 19"" 5:4 Square IPS Monito...",$329.00
3,"EIZO FlexScan EV2456 24.1"" 16:10 IPS Monitor ...",$547.33
4,"EIZO FlexScan EV2760 27"" 16:9 IPS Monitor wit...",$989.00
5,"EIZO FlexScan EV2795FX-BK 27"" 16:9 Docking IP...","$1,069.00"
6,"EIZO FlexScan EV3285 31.5"" 16:9 IPS Monitor (...","$1,839.00"
7,"EIZO FlexScan EV3895FX-BK 37.5"" 24:10 Ultrawi...","$1,889.00"
8,"EIZO FlexScan S1934 19"" 5:4 Square IPS Monito...",$329.00
9,"EIZO FlexScan EV2456 24.1"" 16:10 IPS Monitor ...",$547.33


In [13]:
# best practice to test your class
# if your script is exported as a module,
# the following part is ignored
# this is equivalent to main() in Java

if __name__ == "__main__":

    print("========Test Q1=========")
    domain_name = 'http://www.pricegrabber.com'
    page_url = 'http://www.pricegrabber.com/computer-monitors/browse/'

    products = getProducts(page_url, domain_name, limit=3)
    print(len(products))
    print(products.head(5))

    print("\n")
    print("========Test Q2=========")
    
    #driver = webdriver.Chrome()
    
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)  # set implict wait

    domain_name = 'http://www.pricegrabber.com'
    page_url = 'http://www.pricegrabber.com/computer-monitors/browse/'

    products = getFilteredProducts(driver, page_url, domain_name,
                                   product_line='FlexScan', limit=3)

    driver.quit()
    print(products)

========Test Q1=========
60
                                               name      price
0  Duracell 76355 - DURACELL HIGH POWER INVERTER...     $49.99
1  TVLogic LVM-170A 17" Full HD SD/HD/3G-SDI Mul...  $1,975.00
2  Planar Systems Universal Height Adjust Stand ...     $69.00
3  ikan VXF2-V2 4K HDMI/3G-SDI On-Camera Monitor...    $699.99
4  Lenovo ThinkVision Creator Extreme 27" UHD Mi...  $3,534.00


========Test Q2=========
                                                name      price
0   EIZO FlexScan EV3285 31.5" 16:9 IPS Monitor (...  $1,839.00
1   EIZO FlexScan EV3895FX-BK 37.5" 24:10 Ultrawi...  $1,889.00
2   EIZO FlexScan S1934 19" 5:4 Square IPS Monito...    $329.00
3   EIZO FlexScan EV2456 24.1" 16:10 IPS Monitor ...    $547.33
4   EIZO FlexScan EV2760 27" 16:9 IPS Monitor wit...    $989.00
5   EIZO FlexScan EV2795FX-BK 27" 16:9 Docking IP...  $1,069.00
6   EIZO FlexScan EV3285 31.5" 16:9 IPS Monitor (...  $1,839.00
7   EIZO FlexScan EV3895FX-BK 37.5" 24:10 Ultrawi...  $